# Import Libraires 

In [1]:
import numpy as np
import tensorflow as tf
import keras
import cv2
from keras.models import Sequential
from keras.preprocessing.image import img_to_array
import os
from tqdm import tqdm
import re
import pickle
import matplotlib.pyplot as plt

# Load Data

In [2]:
with open("../input/weather-data/data.pkl", "rb") as f:
    l = pickle.load(f)

In [3]:
l = np.array(l)

In [4]:
    np.random.shuffle(l)

In [13]:
def mean_conv(img):
    size = 6
    stride = 5
    ret = np.ones((3,3))
    stx=0
    for i in range(3):
        sty = 0
        for j in range(3):
            ret[i][j] = np.mean(img[stx:stx+size, sty:sty+size])
            sty += stride
        stx += stride
    
    return ret

In [14]:
SIZE = 16
high_img = []
for i in tqdm(range(len(l))):
    img = l[i]
    img[np.isnan(img)]=np.mean(img[~np.isnan(img)])    
    #resizing image
    img = cv2.resize(img, (SIZE, SIZE))
    high_img.append(img)
temp_img = []
low_img = []
for i in tqdm(range(len(l))):
    img = l[i]
    img[np.isnan(img)]=np.mean(img[~np.isnan(img)])    
    #resizing image
    img = cv2.resize(img, (SIZE, SIZE))
    temp = mean_conv(img)
    temp_img.append(temp)
    img = cv2.resize(temp, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)
    low_img.append(img)

# Data Visualization

In [15]:
for i in range(4):
    a = np.random.randint(0,3600)
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(high_img[a])
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(low_img[a])
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.title('temp Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(temp_img[a])
    plt.axis('off')

# Slicing and Reshaping Images

In [16]:
train_high_image = high_img[:2304]
train_low_image = low_img[:2304]
train_high_image = np.reshape(train_high_image,(len(train_high_image),SIZE,SIZE,1))
train_low_image = np.reshape(train_low_image,(len(train_low_image),SIZE,SIZE,1))

validation_high_image = high_img[2304:2880]
validation_low_image = low_img[2304:2880]
validation_high_image= np.reshape(validation_high_image,(len(validation_high_image),SIZE,SIZE,1))
validation_low_image = np.reshape(validation_low_image,(len(validation_low_image),SIZE,SIZE,1))


test_high_image = high_img[2880:]
test_low_image = low_img[2880:]
test_high_image= np.reshape(test_high_image,(len(test_high_image),SIZE,SIZE,1))
test_low_image = np.reshape(test_low_image,(len(test_low_image),SIZE,SIZE,1))

print("Shape of training images:",train_high_image.shape)
print("Shape of test images:",test_high_image.shape)
print("Shape of validation images:",validation_high_image.shape)


# Defining Model

In [17]:
from keras import layers
def down(filters , kernel_size, apply_batch_normalization = True):
    downsample = tf.keras.models.Sequential()
    downsample.add(layers.Conv2D(filters,kernel_size,padding = 'same', strides = 2))
    if apply_batch_normalization:
        downsample.add(layers.BatchNormalization())
    downsample.add(keras.layers.LeakyReLU())
    return downsample


def up(filters, kernel_size, dropout = False):
    upsample = tf.keras.models.Sequential()
    upsample.add(layers.Conv2DTranspose(filters, kernel_size,padding = 'same', strides = 2))
    if dropout:
        upsample.dropout(0.2)
    upsample.add(keras.layers.LeakyReLU())
    return upsample

def model():
    inputs = layers.Input(shape= [SIZE,SIZE,1])
    d1 = down(128,(3,3),False)(inputs)
    d2 = down(128,(3,3),False)(d1)
    d3 = down(256,(3,3),True)(d2)
    d4 = down(512,(3,3),True)(d3)
    
    #upsampling
    u2 = up(256,(3,3),False)(d4)
    u2 = layers.concatenate([u2,d3])
    u3 = up(128,(3,3),False)(u2)
    u3 = layers.concatenate([u3,d2])
    u4 = up(128,(3,3),False)(u3)
    u4 = layers.concatenate([u4,d1])
    u5 = up(3,(3,3),False)(u4)
    u5 = layers.concatenate([u5,inputs])
    output = layers.Conv2D(1,(2,2),strides = 1, padding = 'same')(u5)
    return tf.keras.Model(inputs=inputs, outputs=output)


model = model()
model.summary()

# Compile 

In [21]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), loss = 'mean_absolute_error',
              metrics = ['acc'])

# Fitting model

In [22]:
model.fit(train_low_image, train_high_image, epochs = 40, batch_size = 16,
          validation_data = (validation_low_image,validation_high_image))

# Prediction Visualization

In [20]:
def plot_images(high,low,predicted):
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.title('High Image', color = 'green', fontsize = 20)
    plt.imshow(high)
    plt.subplot(1,3,2)
    plt.title('Low Image ', color = 'black', fontsize = 20)
    plt.imshow(low)
    plt.subplot(1,3,3)
    plt.title('Predicted Image ', color = 'Red', fontsize = 20)
    plt.imshow(predicted)
   
    plt.show()

for i in range(100,110):
    
    predicted = np.clip(model.predict(test_low_image[i].reshape(1,SIZE, SIZE,1)),0.0,1.0).reshape(SIZE, SIZE,1)
    plot_images(test_high_image[i],test_low_image[i],predicted)

# Saving model

In [9]:
model.save("final_model.h5")

## Thank You !!!